In [1]:
import requests

# Base URL for the DnD 5e API
BASE_URL = "https://www.dnd5eapi.co"

def get_monsters():
    endpoint = "/api/monsters"
    response = requests.get(BASE_URL + endpoint)
    if response.status_code == 200:
        data = response.json()
        return data['results']  # List of monsters with 'name' and 'url'
    else:
        raise Exception(f"Failed to fetch monsters: {response.status_code}")

monsters = get_monsters()
print(f"Total monsters fetched: {len(monsters)}")


Total monsters fetched: 334


In [2]:
import time

def get_monster_details(monster_url):
    response = requests.get(BASE_URL + monster_url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch {monster_url}: {response.status_code}")
        return None

# Collect all detailed monster data
detailed_monsters = []
for monster in monsters:
    details = get_monster_details(monster['url'])
    if details:
        detailed_monsters.append(details)
    time.sleep(0.1)  # To be polite to the API server

print(f"Detailed monsters fetched: {len(detailed_monsters)}")


Detailed monsters fetched: 334


In [3]:
import pandas as pd

# Function to extract required fields from each monster's data
def extract_monster_data(monster):
    data = {}
    # Simple fields
    data['name'] = monster.get('name')
    data['size'] = monster.get('size')
    data['type'] = monster.get('type')
    data['alignment'] = monster.get('alignment')
    data['hit_points'] = monster.get('hit_points')
    data['languages'] = monster.get('languages')
    data['challenge_rating'] = monster.get('challenge_rating')
    data['proficiency_bonus'] = monster.get('proficiency_bonus')
    data['xp'] = monster.get('xp')
    
    # Armor Class
    armor_classes = monster.get('armor_class', [])
    # We'll concatenate types and values
    ac_list = [f"{ac['type']}:{ac['value']}" for ac in armor_classes]
    data['armor_class'] = "; ".join(ac_list)
    
    # Speed
    speed = monster.get('speed', {})
    # Concatenate available speeds
    speeds = [f"{k}:{v}" for k, v in speed.items()]
    data['speed'] = "; ".join(speeds)
    
    # Attributes
    abilities = ['strength', 'dexterity', 'constitution', 'intelligence', 'wisdom', 'charisma']
    for ability in abilities:
        data[ability] = monster.get(ability)
    
    # Proficiencies
    proficiencies = monster.get('proficiencies', [])
    # Concatenate proficiency names and values
    prof_list = [f"{prof['proficiency']['name']}:{prof['value']}" for prof in proficiencies]
    data['proficiencies'] = "; ".join(prof_list)
    
    # Damage Vulnerabilities, Resistances, Immunities
    data['damage_vulnerabilities'] = "; ".join(monster.get('damage_vulnerabilities', []))
    data['damage_resistances'] = "; ".join(monster.get('damage_resistances', []))
    data['damage_immunities'] = "; ".join(monster.get('damage_immunities', []))
    
    # Condition Immunities
    condition_immunities = monster.get('condition_immunities', [])
    # Concatenate condition names
    cond_immunities = [cond['name'] for cond in condition_immunities]
    data['condition_immunities'] = "; ".join(cond_immunities)
    
    # Senses
    senses = monster.get('senses', {})
    # Concatenate senses and their values
    senses_list = [f"{k}:{v}" for k, v in senses.items()]
    data['senses'] = "; ".join(senses_list)
    
    # Special Abilities
    special_abilities = monster.get('special_abilities', [])
    # Concatenate special ability names
    special_abilities_list = [sa['name'] for sa in special_abilities]
    data['special_abilities'] = "; ".join(special_abilities_list)
    
    return data

# Extract data for all monsters
monsters_data = [extract_monster_data(monster) for monster in detailed_monsters]

# Create DataFrame
df = pd.DataFrame(monsters_data)

# Display the first few rows
print(df.head())


                 name    size        type      alignment  hit_points  \
0             Aboleth   Large  aberration    lawful evil         135   
1             Acolyte  Medium    humanoid  any alignment           9   
2  Adult Black Dragon    Huge      dragon   chaotic evil         195   
3   Adult Blue Dragon    Huge      dragon    lawful evil         225   
4  Adult Brass Dragon    Huge      dragon   chaotic good         172   

                           languages  challenge_rating  proficiency_bonus  \
0     Deep Speech, telepathy 120 ft.             10.00                  4   
1  any one language (usually Common)              0.25                  2   
2                   Common, Draconic             14.00                  5   
3                   Common, Draconic             16.00                  5   
4                   Common, Draconic             13.00                  5   

      xp armor_class  ... intelligence  wisdom  charisma  \
0   5900  natural:17  ...           18      

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# Select features and target
# Exclude 'challenge_rating' from features
features = df.drop(['challenge_rating', 'name'], axis=1)  # 'name' is typically not useful for prediction
target = df['challenge_rating']

# Handle missing values if any
features = features.fillna('')  # Simple strategy; you can choose more sophisticated methods

# Identify categorical and numerical columns
# For simplicity, let's assume:
# - Numerical columns: strength, dexterity, constitution, intelligence, wisdom, charisma, hit_points, proficiency_bonus, xp
# - Categorical columns: size, type, alignment, armor_class, speed, proficiencies, damage_vulnerabilities, damage_resistances, damage_immunities, condition_immunities, senses, languages, special_abilities

numerical_features = ['strength', 'dexterity', 'constitution', 'intelligence', 'wisdom', 'charisma', 'hit_points', 'proficiency_bonus', 'xp']
categorical_features = [col for col in features.columns if col not in numerical_features]



In [5]:
# Define preprocessing for numerical and categorical data
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Create preprocessing and training pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Train the model
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2}")

Mean Squared Error: 3.122401891719053
R² Score: 0.9271784382239454


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Update the pipeline with a Random Forest Regressor
model_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Define hyperparameters for tuning
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [None, 10, 20],
    'regressor__min_samples_split': [2, 5],
}

# Setup GridSearch
grid_search = GridSearchCV(model_rf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Train the model
grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV MSE: {-grid_search.best_score_}")

# Predict on test set with the best estimator
y_pred_rf = grid_search.best_estimator_.predict(X_test)

# Evaluate the model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest Mean Squared Error: {mse_rf}")
print(f"Random Forest R² Score: {r2_rf}")


Best parameters: {'regressor__max_depth': None, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 200}
Best CV MSE: 0.05684622434049615
Random Forest Mean Squared Error: 0.7941075209888062
Random Forest R² Score: 0.9814795942668744


In [11]:
def predict_challenge_rating(monster_attributes):
    """
    monster_attributes: dict containing the same keys as features columns
    """
    # Convert to DataFrame
    input_df = pd.DataFrame([monster_attributes])
    
    # Predict
    predicted_cr = grid_search.best_estimator_.predict(input_df)
    return predicted_cr[0]

# Example usage
new_monster = {
    'size': 'Medium',
    'type': 'dragon',
    'alignment': 'chaotic evil',
    'armor_class': 'natural:19',
    'speed': 'walk:40; fly:80',
    'strength': 27,
    'dexterity': 10,
    'constitution': 25,
    'intelligence': 16,
    'wisdom': 15,
    'charisma': 19,
    'hit_points': 546,
    'proficiency_bonus': 8,
    'xp': 390000,
    'proficiencies': 'Perception:10; Stealth:5',
    'damage_vulnerabilities': '',
    'damage_resistances': 'fire',
    'damage_immunities': '',
    'condition_immunities': 'frightened',
    'senses': 'darkvision:120; passive_perception:20',
    'languages': 'Common, Draconic',
    'special_abilities': 'Legendary Resistance; Frightful Presence'
}

predicted_cr = predict_challenge_rating(new_monster)
print(f"Predicted Challenge Rating: {predicted_cr}")


Predicted Challenge Rating: 22.35


In [14]:
# Manual CR Calc Info
def calculate_manual_cr(monster_attributes):
    """
    Calculate the manual CR for a monster using its attributes.
    """
    # Extract relevant attributes
    hit_points = monster_attributes.get('hit_points', 0)
    armor_class = monster_attributes.get('armor_class', 0)
    attack_bonus = monster_attributes.get('attack_bonus', 0)
    damage_per_round = monster_attributes.get('damage_per_round', 0)
    save_dc = monster_attributes.get('save_dc', 0)
    
    # Defensive CR calculation
    if hit_points < 1:
        raise ValueError("Hit Points must be greater than zero for manual CR calculation.")
    base_defensive_cr = hit_points
    if armor_class > 0:
        base_defensive_cr += armor_class  # Incorporate AC scaling
    
    # Offensive CR calculation
    base_offensive_cr = attack_bonus + damage_per_round
    if save_dc > 0:
        base_offensive_cr += save_dc  # Incorporate DC scaling
    
    # Combine defensive and offensive CRs
    manual_cr = (base_defensive_cr + base_offensive_cr) / 2
    
    return manual_cr

